In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from memgpt import Admin
from memgpt import create_client
load_dotenv()

True

# Synthetic LLM Psychiatric Note

### This function will our way of generating data for our Psychiatrist Agent.



In [2]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

def synthesize_note(pathology_A, pathology_B, note_type, note='', memory=''):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": f"You are a Psychiatrist in practice. You monitor a patient's disease/disorder progression for {pathology_A} with \
                                            unknown, underlying {pathology_B}. Please do not disclose the secondary pathology in your notes. \
                                            Unless there was evidence confirming the discovery of the secondary diagnosis in your memory or previous note, you currently do not know about the secondary pathology. \
                                            Please, refer to patient as 'client'. Be obvservant of whether the patient's condition is improving or worsening."},
                                            
            {"role": "user", "content": f"Synthesize a {note_type} note for your patient that could reflect a real-world {note_type} note (you are not writing a template, \
                                          your response should represent a potential case-study). The contents of the note should not be too generic, reflecting the authenticity of every individual's experiences.\n \
                                          Please include a date, and which is newer than the dates provided in the last note and/or memory.\n \
                                          Here is the last note you wrote from the previous visit: \n{note}\n \
                                          Here is a collection of your memory about this patient: \n{memory}"}
        ]
    )
    return completion

## **Psychiatrist's Psychiatric Interview Note**

In [3]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "psychiatric interview"
note = "N/A"
memory = "N/A"

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: August 15, 2021

Client presented today for a follow-up session regarding their ongoing struggles with symptoms of Post-Traumatic Stress Disorder (PTSD), including recurring flashbacks and nightmares related to a traumatic event in their past. Client appeared visibly tense and anxious upon arrival, fidgeting with their hands and avoiding direct eye contact.

During our session, client shared that their sleep patterns have been severely disrupted in recent weeks, with frequent episodes of insomnia and difficulty staying asleep once they do manage to drift off. Client reported feeling constantly exhausted and irritable due to the lack of restful sleep, impacting their ability to focus and engage in daily activities.

Client described feeling overwhelmed by intrusive thoughts during the night, causing them to jolt awake in a state of panic. The fear of experiencing these distressing episodes has further exacerbated their anxiety surrounding bedtime, creating a cycle of insomnia and 

### **For now, read through the note to ensure the second pathology is not exposed.**

### Run MemGPT Server with
`memgpt server [--debug]`

In [4]:
admin = Admin(base_url="http://localhost:8283", token="lPEpHarXb_ZE8QOJfs2PDg")

In [5]:
response = admin.create_user()
user_id = response.user_id # unique UUID for the user 
api_key = response.api_key # bearer token for authentication

{'user_id': '6ef4aa42-39dc-45b6-bd4e-e388d9cc3fb2', 'api_key': 'sk-57e9718ead8d286a93bbc8a5c8598683f96aad8e900d03dd'}


In [6]:
admin.get_users()

GetAllUsersResponse(user_list=[{'user_id': '90cf6779-da7d-42cb-87db-46157cb2fc31'}, {'user_id': '00000000-0000-0000-0000-2c8db1d93428'}, {'user_id': 'd6497bda-1ca3-4cc1-849e-620fd091a5a5'}, {'user_id': '0db40444-4dc4-4d1d-897b-7d5b6aa9dbea'}, {'user_id': 'e825629f-38e1-4da0-89c8-2f59f09effad'}, {'user_id': 'af63b288-2c61-4256-81c3-40f7b2dace81'}, {'user_id': '6ef4aa42-39dc-45b6-bd4e-e388d9cc3fb2'}])

In [7]:
for ul in admin.get_users().user_list:
    admin.get_keys(ul['user_id'])

{"api_key_list":["sk-54658101914c3c4310eadcbef47a4fbe53bcd1c7148aef67"]} 200
{"api_key_list":[]} 200
{"api_key_list":["sk-6446a604e8a89485e1db204a031600e699cfa4b6c1a952e9"]} 200
{"api_key_list":["sk-8ded11141905f70c1126b8ec285850dda6c934fda012e92c"]} 200
{"api_key_list":["sk-771c6965e6d273ed8d453982d05a3098930dc4c454bdf061"]} 200
{"api_key_list":["sk-c350f59fefe691b34543264a24d210796c4f53426ec8e7f2"]} 200
{"api_key_list":["sk-57e9718ead8d286a93bbc8a5c8598683f96aad8e900d03dd"]} 200


In [10]:
client_memgpt = create_client(base_url="http://localhost:8283", token="sk-54658101914c3c4310eadcbef47a4fbe53bcd1c7148aef67")
# client_memgpt = create_client(base_url="http://localhost:8283", token="sk-6446a604e8a89485e1db204a031600e699cfa4b6c1a952e9")

In [16]:
psychiatrist_agent = client_memgpt.create_agent(
    name="Psychiatrist Agent",
    persona="You are a friendly psychiatrist and therapist. \
             You have authority to counsel patients, prescribe medication, order laboratory tests, order neuroimaging, conduct physical examinations, provide therapy when you see fit.",
    human="Youself. You are talking/thinking to youself about the best course of action for your patients."
)

In [17]:
client_memgpt.list_agents()

ListAgentsResponse(num_agents=1, agents=[{'id': 'b030c919-8d9e-4dc5-93f5-46c1e85eb118', 'name': 'Psychiatrist Agent', 'human': 'basic', 'persona': 'sam_pov', 'created_at': '2024-04-24T05:18:36.028315', 'tools': [{'name': 'archival_memory_insert', 'tags': ['memgpt-base'], 'source_type': 'python', 'source_code': 'def archival_memory_insert(self: Agent, content: str) -> Optional[str]:\n    """\n    Add to archival memory. Make sure to phrase the memory contents such that it can be easily queried later.\n\n    Args:\n        content (str): Content to write to the memory. All unicode (including emojis) are supported.\n\n    Returns:\n        Optional[str]: None is always returned as this function does not produce a response.\n    """\n    self.persistence_manager.archival_memory.insert(content)\n    return None\n', 'json_schema': {'name': 'archival_memory_insert', 'description': 'Add to archival memory. Make sure to phrase the memory contents such that it can be easily queried later.', 'par

In [18]:
agent_name = 'Psychiatrist Agent'
agent_id   = [agents['id'] for agents in client_memgpt.list_agents().agents if agents['name'] == agent_name][0]
agent_id

'b030c919-8d9e-4dc5-93f5-46c1e85eb118'

In [15]:
client_memgpt.delete_agent(agent_id)

In [19]:
psychiatrist_agent = client_memgpt.get_agent(agent_id=agent_id, agent_name=agent_name)

In [20]:
# Create a helper that sends a message and prints the assistant response only
def send_message(agent_id, message: str):
    """
    sends a message and prints the assistant output only.
    :param message: the message to send
    """
    response = client_memgpt.user_message(agent_id=agent_id, message=message)
    for r in response:
        # Can also handle other types "function_call", "function_return", "function_message"
        if "assistant_message" in r:
            print("ASSISTANT:", r["assistant_message"])
        elif "internal_monologue" in r:
            print("THOUGHTS:", r["internal_monologue"])
    return response

In [21]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "Let's review this information for insightful details. The patient exhibits significant insomnia, caused by recurrent PTSD symptoms and anxiety. Despite previous attempts at relaxation and improved sleep hygiene, these issues persist, leaving the patient feeling frustrated. Cognitive behavior therapy techniques with a focus on managing insomnia seems a suitable next step. In turn, this might hopefully reduce both nighttime anxiety and intrusive thoughts. To properly reflect and engage in analysis, I will pen down these details in my archival memory.",
  'id': 'e4fc9800-8a79-45ba-a265-f2b08aab1a60',
  'date': '2024-04-24T05:18:51.745015+00:00'},
 {'function_call': "archival_memory_insert({'content': 'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cogniti

In [22]:
response.messages[1]['function_call']

"archival_memory_insert({'content': 'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.'})"

In [23]:
import re

def insert_memory(response):
    memory = []
    for r in response.messages:
        if not 'function_call' in r:
            continue
        if not 'archival_memory' in r['function_call'] or 'core_memory' in r['function_call']:
            continue
        content = r['function_call']
        start = re.search("content(\')?: ", content).end()
        end   = re.search("}\)", content).start()
        memory.append(content[start:end-1])
    if memory:
        return memory[0]
    else:
        return ''

In [24]:
insert_memory(response)

"'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021."

In [25]:
archival_memory = []
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021."]

## **FIRST Progress Note & Psychiatrist Reflection and Memory**

In [26]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: October 15, 2022

Client presented for their follow-up appointment today. They reported experiencing increasing difficulty falling asleep for the past two weeks. Client mentioned feeling on edge due to vivid nightmares that have intensified in frequency, leading to disrupted sleep patterns. They shared feeling exhausted and irritable during the day, struggling to concentrate at work. Client has been using relaxation techniques learned during therapy sessions; however, they expressed frustration at the lack of improvement in their sleep quality.

During the session, client displayed signs of hyperarousal, including fidgeting and a heightened startle response when discussing the traumatic event. Client's affect appeared anxious, and their speech was rapid, indicating underlying distress. Client acknowledged feeling overwhelmed by intrusive thoughts related to the trauma, affecting their daily functioning.

We discussed the importance of maintaining a consistent sleep routine and av

In [27]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The client's sleep issues have intensified, leading to nightmares and difficulties in daytime functioning, as well as increasing signs of hyperarousal. The relaxation techniques previously provided have not delivered the desired improvements. Mindfulness practices were advised as an additional approach, in hopes of calming the mind before sleep time. Tracking these modifications will be beneficial to assess progress, so I'll document these developments in my archival memory. The situation needs to be observed closely given the decline in sleep quality despite stability in PTSD symptoms. The next follow-up is in two weeks.",
  'id': '975edcf6-e724-480a-b3ae-7faed9bd199f',
  'date': '2024-04-24T05:19:22.742819+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impactin

In [28]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **SECOND Progress Note & Psychiatrist Reflection and Memory**

In [29]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: May 25, 2023

Client presented today with noticeable improvements since our last visit. They reported a decrease in the frequency and intensity of nightmares related to past traumatic experiences, which is a positive development in managing their PTSD symptoms. Client mentioned that implementing relaxation techniques before bedtime has helped them fall asleep faster and experience fewer disruptions during the night.

During the session, client seemed more at ease discussing their emotions and thoughts, demonstrating progress in building trust and rapport in our therapeutic relationship. They expressed a growing awareness of their triggers and have been actively practicing grounding exercises to manage overwhelming feelings of anxiety and distress.

Client also shared that they have been gradually incorporating a structured bedtime routine, including turning off screens an hour before sleep and creating a calming environment in their bedroom. While client still experiences occasio

In [30]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "A positive shift is evident in the client's situation. They are demonstrating improvement with a decrease in nightmare frequency and intensity, faster sleep onset, fewer sleep disruptions, and enhanced emotional discussions. Their rigorous practice of relaxation techniques and maintaining a structured bedtime routine have played a significant positive role in their progress. Their resilience and commitment are worth appreciating. This is promising progress but requires ongoing support. Let's keep these positive developments in the archival memory for future reference.",
  'id': '87889a42-9178-4e38-92c9-6e3ce48b990b',
  'date': '2024-04-24T05:19:48.248953+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On May 25, 2023, the client demonstrated remarkable improvement. There was a reduction in the frequency and intensity of PTSD related nightmares and less difficulty falling asleep, owing to systematic implementation of relaxation techniques and 

In [31]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **THIRD Progress Note & Psychiatrist Reflection and Memory**

In [32]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: August 25, 2021

Client presented today with continued symptoms of insomnia and anxiety related to past traumatic experiences. Client reports that despite attempting relaxation techniques before bed, they still struggle to fall asleep and often wake up multiple times throughout the night. Client describes persistent nightmares, which contribute to their overall feelings of distress and exhaustion.

During today's session, we explored potential triggers for client's insomnia and anxiety, focusing on recent stressors at work and difficulties in personal relationships. Client expressed feelings of helplessness and heightened arousal when faced with conflict, indicating a possible exacerbation of their PTSD symptoms.

Client appears fatigued and irritable, displaying signs of increased emotional reactivity compared to previous sessions. Their overall demeanor suggests a decline in mental well-being and a need for additional support in managing their symptoms.

Client's PTSD symptoms 

In [33]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The client exhibits heightened symptoms, constant wakefulness, persistent nightmares, and a pervasive sense of helplessness. Their PTSD symptoms have reportedly increased, affecting their daily stress management and sleep patterns. However, the client is encouraged to continue therapy sessions — focusing on trauma processing and mindfulness-based practices for the insomnia. This situation calls for a careful reevaluation of the client's treatment plan as these heightened indicators can detrimentally impact the recovery process. It's pertinent to document these changes in archival memory for future reference.",
  'id': '4cccd1ba-f5ae-4c86-b862-d46f7c4bd013',
  'date': '2024-04-24T05:20:57.875169+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On August 25, 2021, the client showed heightened symptoms of insomnia and anxiety linked to previous traumatic experiences. Their PTSD symptoms seemingly worsened, affecting daily stress management and le

In [34]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **FOURTH Progress Note & Psychiatrist Reflection and Memory**

In [35]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: February 15, 2023

Client presented today appearing more fatigued than usual, reporting continued difficulty falling asleep despite practicing good sleep hygiene. Client shared experiencing intrusive memories related to a traumatic event, leading to heightened anxiety and reactivity. Client mentioned a slight increase in nightmares, with themes of helplessness and danger persisting. 

During the session, client appeared visibly distressed when discussing specific triggers, leading to increased heart rate and shallow breathing. Client expressed frustration at feeling constantly on edge and struggling to concentrate on daily tasks. Client showed decreased interest in hobbies previously enjoyed and displayed signs of emotional numbing during the session.

Client's PTSD symptoms appear to be worsening, impacting sleep quality and overall functioning. We will explore tailored interventions to address both the PTSD symptoms and sleep disturbances in the upcoming sessions to improve cli

In [36]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "There's a marked increase in the client's symptoms including fatigue, difficulty in falling asleep, intrusive memories, and heightened reactivity. Emotional numbing and decreased interest in hobbies present an alarming retreat from positive engagements. It's crucial to revisit intervention strategies targeting both PTSD symptoms and sleep disturbances. Upcoming sessions must work on enriched coping mechanisms and immediate supportive measures. The heightened symptoms and client's distress necessitate a prompt and sensitive addressal. Plan to keep these significant details in archival memory for future reference.",
  'id': 'c171e91e-c523-49bb-bbc3-57ffad1868eb',
  'date': '2024-04-24T05:21:57.635854+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On February 15, 2023, the client\'s condition was noted to be deteriorating, with increased symptoms of fatigue, sleep disturbances, intrusive memories, and heightened reactivity. A characteristic ret

In [37]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **FIFTH Progress Note & Psychiatrist Reflection and Memory**

In [38]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: September 15, 2021

Client presented today with ongoing symptoms of hyperarousal, including increased irritability, difficulty concentrating, and hypervigilance. Client reported persistent nightmares related to past traumatic experiences, causing disruptions in sleep patterns. Client shared feeling overwhelmed by intrusive thoughts and memories, contributing to heightened anxiety levels throughout the day.

Client demonstrated some improvement in utilizing relaxation techniques discussed in previous sessions to manage anxiety symptoms. Client reported engaging in regular exercise routine, which has helped in reducing overall stress levels. However, client expressed frustration over continued struggles with falling asleep and staying asleep, revealing persistent insomnia symptoms.

We discussed the importance of maintaining a consistent sleep schedule and implementing good sleep hygiene practices. Client agreed to keep a sleep diary to track patterns and identify any triggers cont

In [39]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The session from September 15, 2021, brought mixed results. On one hand, the client exhibited persistent insomnia symptoms, nightmares, and hyperarousal. On the other hand, anxiety management showed improvement through the use of relaxation techniques and regular exercise. This is a critical juncture in the treatment where the client is open to exploring pharmacological interventions if necessary. It's important to remember that progress is not linear, and the client's frustrations reflect normal reactions to complex recovery processes. I must archive these therapy milestones for tracking long-term progress.",
  'id': 'c323a22b-2025-4b3c-b769-0b061902dd0b',
  'date': '2024-04-24T05:24:05.040978+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On September 15, 2021, the client struggled with ongoing nightmares, hyperarousal, and persistent insomnia, despite improvements in anxiety management through relaxation techniques and regular exercise. T

In [40]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **SIXTH Progress Note & Psychiatrist Reflection and Memory**

In [41]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: October 15, 2021

Client presented today with noticeable improvements in mood compared to the last visit. They reported a slight decrease in the frequency of intrusive thoughts related to past traumatic events, indicating a positive response to the trauma-focused therapy sessions. Client also mentioned feeling more in control of their emotions and expressed a growing awareness of triggers, showcasing progress in emotional regulation.

During the session, client shared that their sleep patterns have shown some signs of improvement. While they continue to experience difficulties falling asleep due to racing thoughts, they reported fewer instances of waking up in the middle of the night. Client attributed this progress to the sleep hygiene techniques discussed in previous sessions. It is encouraging to see these strategies yielding some positive results.

Client stated that they have been actively engaging in relaxation techniques such as deep breathing exercises and guided imagery 

In [42]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The October 15, 2021 session brought encouragement, showing the client's progress in mood, sleep patterns, emotional regulation, and anxiety management. Mental health is a complex field, wherein recovery is a non-linear journey full of highs and lows. This session, however, highlighted the highs, showcasing promising improvements. The continued patience, endurance, and enhanced self-awareness of the client are proof of their resilience. This point of progress in my client's therapy creates an optimistic benchmark in their recovery narrative. I must ensure these details are saved in the archival memory.",
  'id': '5f5bd020-7a3a-4f25-a30e-0582832af650',
  'date': '2024-04-24T05:26:40.489310+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "The October 15, 2021 session showed encouraging progress in the client\'s therapy. They exhibited improved mood, better emotional regulation, progress in managing their anxiety, and some improvement in their sl

In [43]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **SEVENTH Progress Note & Psychiatrist Reflection and Memory**

In [44]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: August 15, 2022

Client presented with increased agitation and restlessness during today's session. They reported persistent difficulties falling asleep and staying asleep, endorsing an average of 4 hours of sleep per night over the past week. Client described frequent nightmares related to past traumatic experiences, contributing to their overall sense of hypervigilance and anxiety.

During the session, client displayed signs of emotional distress when discussing specific trauma triggers, leading to heightened physiological arousal, including rapid heart rate and shallow breathing. Client appeared fatigued and expressed frustration about the impact of sleep disturbances on their daily functioning.

Client's PTSD symptoms seem to have intensified since our last session, with an apparent escalation in intrusive thoughts and avoidance behaviors. Client's overall mood appeared more subdued, and they reported feeling increasingly detached from their surroundings.

Treatment plan adju

In [45]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The situation took a turn on August 15, 2022. The intensification of PTSD symptoms, escalating to heightened anxiety, nightmares, increased agitation, and recurrence of insomnia were hard to see in the client. The emotional distress observed, especially when discussing specific trauma triggers, hints at the need to revisit and potentially intensify certain aspects of our therapeutic strategies. It's solacing to know that the client was open to changing their treatment plan and exploring new options. While periods like this are challenging, they are part of the journey. I must take care to save these moments in the archival memory to offer a comprehensive view of the therapeutic process.",
  'id': '24f9c256-034c-40f4-af9f-8a4da76e1c03',
  'date': '2024-04-24T05:27:44.660520+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "On August 15, 2022, the client\'s session revealed exacerbating PTSD symptoms. They struggled with increased agitation, esca

In [46]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **EIGHTH Progress Note & Psychiatrist Reflection and Memory**

In [47]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: September 15, 2023

Client presented for their follow-up appointment today. Client reported experiencing increased difficulty falling asleep at night over the past two weeks. They described feeling more on edge during the day, with heightened levels of anxiety and hypervigilance. Client shared that they have been experiencing more frequent nightmares, which have been disrupting their sleep even further.

During the session, client displayed signs of restlessness and emotional distress, frequently fidgeting and avoiding eye contact. Client's affect appeared more subdued compared to the last session, and they exhibited signs of fatigue and exhaustion.

Client mentioned that they have been trying relaxation techniques before bed, such as deep breathing and progressive muscle relaxation, but have found limited success in improving their sleep quality. Client expressed frustration and helplessness regarding their inability to get adequate rest, which has been impacting their daily fun

In [48]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The September 15, 2023, session marked a critical point in the client's therapy journey. They had increasing difficulty sleeping which seemed to exacerbate other PTSD symptoms. Notably, the client expressed feelings of frustration and helplessness about the ongoing sleep disruption impacting daily functioning. Even attempts to apply relaxation techniques before bed seemed to offer limited relief. Nonetheless, their willingness to maintain a regular sleep schedule, implement healthy sleep hygiene practices, explore cognitive-behavioral therapy for insomnia, and acknowledge a multidisciplinary approach signals their enduring resilience and motivation for recovery. I should archive this session for future reflections and analysis.",
  'id': 'f056ac5e-f0fd-48f2-8592-efe188986fa6',
  'date': '2024-04-24T05:28:46.419599+00:00'},
 {'function_call': "archival_memory_insert({'content': 'The September 15, 2023 session was a pivotal one. The client revealed increasing sle

In [49]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **NINTH Progress Note & Psychiatrist Reflection and Memory**

In [50]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

Date: 07/15/2023

Client presented today with continued symptoms of hyperarousal and avoidance behaviors consistent with PTSD. They reported increased anxiety levels over the past week, particularly related to loud noises triggering intrusive memories of their traumatic event. Client shared that they have been experiencing worsening nightmares, impacting their quality of sleep and leaving them feeling fatigued during the day.

In terms of insomnia symptoms, client reported persistent difficulties falling asleep and staying asleep throughout the night. They described feeling restless and on edge, leading to increased frustration with their inability to achieve restful sleep. Client confirmed that they have been trying relaxation techniques before bed, but effectiveness has been limited.

Client's mood appeared subdued, and they expressed feelings of hopelessness regarding their recovery journey. Despite this, client demonstrated engagement in therapy sessions and expressed a willingness

In [51]:
# Send a message and see the response
message = f"Please reflect on the information you wrote regarding your client: {completion.choices[0].message.content}"

response = send_message(agent_id, message)
response.messages

[{'internal_monologue': "The July 15, 2023, session illustrated an escalation in the client's symptoms, with increased anxiety levels, heightened hyperarousal, avoidance behaviors, worsening nightmares, and persistent insomnia. Despite these obstacles, their willingness to continue treatment and engage in therapies conveyed commitment. They were open to discussing amendments in their treatment plan, such as increased therapy sessions, introduction of sleep hygiene strategies, challenging avoidance behaviors, and potential medication adjustment. It is important for me to remember that resilience can often bloom amidst despair. This session, although challenging, is another crucial point in the client's recovery journey that should be archived.",
  'id': 'd26132ff-c467-4adf-9f96-16427f187e44',
  'date': '2024-04-24T05:31:04.266891+00:00'},
 {'function_call': 'archival_memory_insert({\'content\': "The session on July 15, 2023, marked an escalation in the client\'s situation. The client re

In [52]:
archival_memory.append(insert_memory(response))
archival_memory

["'On August 15, 2021, the client exhibited symptoms of PTSD including insomnia, intrusive thoughts, and heightened emotional distress. Despite attempting relaxation techniques and good sleep hygiene practices, these symptoms continue to persist. Moving forward, cognitive-behavioral therapy techniques focusing on insomnia management will be incorporated into the treatment plan. Progress will be monitored closely and adjustments will be made as needed. Next appointment is scheduled for September 5, 2021.",
 '"On October 15, 2022, it was noted that the client\'s sleep disturbances had intensified, highlighted by increased difficulty falling asleep and the frequency of nightmares. This was impacting daytime activities, concentration, and causing a heightened sense of hyperarousal. While their engagement in therapy remains positive, relaxation techniques provided earlier didn\'t significantly improve sleep quality. To combat this, mindfulness practices were advised as a non-pharmacological

## **FINAL Progress Note**

In [53]:
pathology_A = "PTSD"
pathology_B = "Primary Insomnia"
note_type = "progress"
note = completion.choices[0].message.content
memory = '\n'.join(archival_memory)

completion = synthesize_note(pathology_A, pathology_B, note_type)
print(completion.choices[0].message.content)

**Date:** June 15, 2022

**Subjective:**
The client presented today feeling more restless and anxious compared to their last visit. They reported increased difficulty falling asleep and staying asleep due to persistent nightmares related to their traumatic experiences. Client shared feeling fatigued and irritable during the day, affecting their ability to concentrate at work. They expressed frustration over the lack of restful sleep impacting their overall quality of life.

**Objective:**
During the session, the client appeared visibly fatigued with dark circles under their eyes. They displayed signs of hyperarousal, such as fidgeting and frequent shifts in their seat. Client's vital signs were within normal limits. They acknowledged a slight increase in alcohol consumption as a coping mechanism for their insomnia.

**Assessment:**
The client's PTSD symptoms appear to be exacerbating their insomnia, leading to significant sleep disruption and impairment in daily functioning. The increa